## Assignment 5

### Question 1

Code to launch complete cloud architecture code and tear it down

In [4]:
import boto3

###########################################################################
##############          (A) LAUNCH CLOUD ARCHITECTURE         #############
###########################################################################

#---------------------(1) CREATE LAMBDA FUNCTION--------------------------

# Access IAM role to allow Lambda to interact with other AWS resources
aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')

# Open zipped directory that contains lambda function
with open('a5.zip', 'rb') as f:
    lambda_zip = f.read() 

try:
    # If function hasn't yet been created, create it
    response = aws_lambda.create_function(
        FunctionName='a5_lambda',
        Runtime='python3.9',
        Role=role['Role']['Arn'], 
        Handler='lambda_function.lambda_handler', 
        Code=dict(ZipFile=lambda_zip),
        Timeout=300
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip file contents
    response = aws_lambda.update_function_code(
        FunctionName='a5_lambda',
        ZipFile=lambda_zip
        )

lambda_arn = response['FunctionArn']
print("Lambda ARN: ", lambda_arn)

#---------------------(2) CREATE S3 BUCKET-------------------------------

# Initialize s3 client and resource
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

# Create S3 bucket
s3.create_bucket(Bucket='mariagabrielaa-a5')

# Get the list_buckets response
response = s3.list_buckets()

# Print each bucket name (to check bucket was created)
for bucket in response['Buckets']:
    print("Bucket name: ", bucket['Name'])


#---------------------(3) CREATE DYNAMO DB TABLE-------------------------

dynamodb = boto3.resource('dynamodb')

table = dynamodb.create_table(
    TableName='a5_results',
    KeySchema=[
        {
            'AttributeName': 'user_id',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'user_id',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
)

# Wait until AWS confirms that table exists before moving on
table.meta.client.get_waiter('table_exists').wait(TableName='a5_results')

# get data about table (should currently be no items in table)
print("Number of current tables: ", table.item_count)
print("Table creation time: ", table.creation_date_time)


###########################################################################
##############         (B) TEAR DOWN CLOUD ARCHITECTURE       #############
###########################################################################

#----------------------(1) DELETE S3 OBJECTS------------------------------

def remove_objects(bucket_name):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)

    for item in bucket.objects.all():
        item.delete()

#---------------------(2) DELETE DYNAMO DB TABLE--------------------------

def delete_table(table_object):
    table_object.delete()

Lambda ARN:  arn:aws:lambda:us-east-1:589049386593:function:a5_lambda
Bucket name:  mariagabrielaa-a5
Number of current tables:  0
Table creation time:  2023-04-20 17:27:13.653000-05:00


### Question 2
Modify Lambda function code to store raw JSON data (i.e. the input payload) for valid survey submissions as a JSON file in S3.

In [5]:
# The updated lambda function is in 'lambda_q2.py' and for reference, it looks like this:

import boto3
import json

s3 = boto3.resource('s3') 

def lambda_handler(event, context):

    key = event["user_id"] + event["timestamp"] + ".json"

    if event["time_elapsed"] <= 3:
        return {
        'statusCode': 400,
        'body': 'Invalid entry. Not enough time elapsed.'
        }
    elif event["freetext"] == "":
        return {
        'statusCode': 400,
        'body':'Invalid entry. No text response'
        }
    else:
        # Only upload valid survey entries to S3 bucket
        s3.Bucket('mariagabrielaa-a5').put_object(Key=key, Body=json.dumps(event))

        return {
        'statusCode': 200,
        'body': 'OK'
        }

### Question 3

For each valid survey submission, modify your Lambda function to insert/update the participant’s record into a DynamoDB database, with the User’s ID number as the partition key, along with their answers to each of the survey questions (including the written response), as well as the number of times the user has completed a survey overall (including the current survey you are processing).



In [19]:
# The updated lambda function is in 'lambda_q3.py' and for reference, it looks like this:
import boto3
import json
from boto3.dynamodb.conditions import Key

s3 = boto3.resource('s3') 
dynamodb = boto3.resource('dynamodb')

def lambda_handler(event, context):

    key = event["user_id"] + event["timestamp"] + ".json"

    if event["time_elapsed"] <= 3:
        return {
        'statusCode': 400,
        'body': 'Invalid entry. Not enough time elapsed.'
        }
    elif event["freetext"] == "":
        return {
        'statusCode': 400,
        'body':'Invalid entry. No text response'
        }
    else:
        # Only upload valid survey entries to S3 bucket
        s3.Bucket('mariagabrielaa-a5').put_object(Key=key, Body=json.dumps(event))

        table = dynamodb.Table("a5_results")
        
        # First, check if there is already an entry in the table for the user_id
        response = table.get_item(
            Key={'user_id': event["user_id"]}
            )
        # If no previous entry, then set num_submission to 0
        if "Item" not in response:
            num_submission = 0
        # Otherwise, query the previous number of submissions
        else:
            response = table.query(KeyConditionExpression=Key('user_id').eq(event["user_id"]),
            ProjectionExpression='num_submission')
            num_submission = response['Items'][0]['num_submission']

        # Insert survey entry in DynamoDb table
        table.put_item(Item= {
                            'user_id': event['user_id'],
                            'q1': event['q1'],
                            'q2': event['q2'],
                            'q3': event['q3'],
                            'q4': event['q4'],
                            'q5': event['q5'],
                            'freetext': event['freetext'],
                            'num_submission': int(num_submission) + 1
                            }
                        )
        return {
        'statusCode': 200,
        'body': 'OK. DynamoDB table updated and response uploaded to S3'
        }

### Question 4

Check whether your Lambda function is working properly. Specifically, you should invoke your function with the following test data input from participants 1, 2, 3, 4, and 5. 

In [7]:
import time

p1_response_1 = {
    "user_id": "0001",
    "timestamp": "092821120000",
    "time_elapsed": 5,
    "q1": 5,
    "q2": 3,
    "q3": 2,
    "q4": 2,
    "q5": 4,
    "freetext": "I had a very bad day today..."
}

# Invoke lambda function
r = aws_lambda.invoke(FunctionName='a5_lambda',
                      InvocationType='RequestResponse',
                      Payload=json.dumps(p1_response_1))
json.loads(r['Payload'].read())

{'statusCode': 200,
 'body': 'OK. DynamoDB table updated and response uploaded to S3'}

In [8]:
time.sleep(10)

p1_response_2 = {
    "user_id": "0001",
    "timestamp": "092821120001",
    "time_elapsed": 2,
    "q1": 5,
    "q2": 3,
    "q3": 2,
    "q4": 2,
    "q5": 4,
    "freetext": "I had a very bad day today..."
}

# Invoke lambda function
r = aws_lambda.invoke(FunctionName='a5_lambda',
                      InvocationType='RequestResponse',
                      Payload=json.dumps(p1_response_2))
json.loads(r['Payload'].read())

{'statusCode': 400, 'body': 'Invalid entry. Not enough time elapsed.'}

In [9]:
time.sleep(10)

p1_response_3 = {
    "user_id": "0001",
    "timestamp": "093021120300",
    "time_elapsed": 5,
    "q1": 1,
    "q2": 1,
    "q3": 2,
    "q4": 2,
    "q5": 2,
    "freetext": "I lost my car keys this afternoon at lunch, so I'm more stressed than normal"
}
# Invoke lambda function
r = aws_lambda.invoke(FunctionName='a5_lambda',
                      InvocationType='RequestResponse',
                      Payload=json.dumps(p1_response_3))
json.loads(r['Payload'].read())

{'statusCode': 200,
 'body': 'OK. DynamoDB table updated and response uploaded to S3'}

In [10]:
time.sleep(10)

p2_response_1 = {
    "user_id": "0002",
    "timestamp": "092821120000",
    "time_elapsed": 5,
    "q1": 4,
    "q2": 1,
    "q3": 1,
    "q4": 1,
    "q5": 3,
    "freetext": "I'm having a great day!"
}

# Invoke lambda function
r = aws_lambda.invoke(FunctionName='a5_lambda',
                      InvocationType='RequestResponse',
                      Payload=json.dumps(p2_response_1))
json.loads(r['Payload'].read())

{'statusCode': 200,
 'body': 'OK. DynamoDB table updated and response uploaded to S3'}

In [11]:

time.sleep(10)

p3_response_1 = {
    "user_id": "0003",
    "timestamp": "092821120001",
    "time_elapsed": 5,
    "q1": 1,
    "q2": 3,
    "q3": 3,
    "q4": 1,
    "q5": 4,
    "freetext": "It was a beautiful, sunny day today."
}

# Invoke lambda function
r = aws_lambda.invoke(FunctionName='a5_lambda',
                      InvocationType='RequestResponse',
                      Payload=json.dumps(p3_response_1))
json.loads(r['Payload'].read())

{'statusCode': 200,
 'body': 'OK. DynamoDB table updated and response uploaded to S3'}

In [12]:
time.sleep(10)

p4_response_1 = {
    "user_id": "0004",
    "timestamp": "092821120002",
    "time_elapsed": 8,
    "q1": 1,
    "q2": 1,
    "q3": 1,
    "q4": 1,
    "q5": 1,
    "freetext": "I had a very bad day today..."
}

# Invoke lambda function
r = aws_lambda.invoke(FunctionName='a5_lambda',
                      InvocationType='RequestResponse',
                      Payload=json.dumps(p4_response_1))
json.loads(r['Payload'].read())

{'statusCode': 200,
 'body': 'OK. DynamoDB table updated and response uploaded to S3'}

In [13]:
time.sleep(10)

p5_response_1 = {
    "user_id": "0005",
    "timestamp": "092821122000",
    "time_elapsed": 5,
    "q1": 3,
    "q2": 3,
    "q3": 3,
    "q4": 3,
    "q5": 3,
   "freetext": "I'm feeling okay, but not spectacular"
}

# Invoke lambda function
r = aws_lambda.invoke(FunctionName='a5_lambda',
                      InvocationType='RequestResponse',
                      Payload=json.dumps(p5_response_1))
json.loads(r['Payload'].read())

{'statusCode': 200,
 'body': 'OK. DynamoDB table updated and response uploaded to S3'}

#### (a)
Print a list of the objects in your S3 bucket. 

In [14]:
# Initialize S3 client
s3 = boto3.client('s3')

# List S3 objects
response = s3.list_objects(Bucket="mariagabrielaa-a5")
lst_objects = []
for obj in response['Contents']:
    lst_objects.append(obj['Key'])

print(lst_objects)


['0001092821120000.json', '0001093021120300.json', '0002092821120000.json', '0003092821120001.json', '0004092821120002.json', '0005092821122000.json']


### (b)

Using a SQL-style query, Query your DynamoDB database to identify all user_ids with a Likert scale response of at least 3 on q2 or q4

In [18]:
response = table.meta.client.execute_statement(
    Statement='''
              SELECT user_id
              FROM a5_results
              WHERE q2 >= 3 OR q4 >= 3
              '''
)

item = response['Items']
print(item)

[{'user_id': '0005'}, {'user_id': '0003'}]


### (c)

Get the DynamoDB data associated with user_id ‘0001’

In [16]:
response = table.get_item(
        Key={'user_id': '0001'}
        )
print(response['Item'])

{'q1': Decimal('1'), 'q2': Decimal('1'), 'user_id': '0001', 'q3': Decimal('2'), 'q4': Decimal('2'), 'q5': Decimal('2'), 'num_submission': Decimal('2'), 'freetext': "I lost my car keys this afternoon at lunch, so I'm more stressed than normal"}


### (d) 

#### Discuss the relative merits of performing partition key lookups (as in (c)) versus more complex analytical queries (as in part (b)) in a DynamoDB database. What is another type of scalable database that we talked about in class that might be a better option if we needed to perform a lot of relatively small, fast analytical queries (as in (b)) on this data (assuming we still need to perform many small insertions and updates as participants continue to submit surveys)? Why might this be a better option?

Dynamo DB partitions the data by using the partition key, in this case `user_id`. This makes partition look ups like in **part c** really fast and efficient. One limitation though is that these lookups can only retrieve data associated with a single partition key, so it is a less suitable approach for performing more complex queries where we can analyze data across several attributes at the same time, like we do in **part b**.

Dynamo DB can handle SQL-type more complex analytical queries (as in **part b**), but only to a degree of complexity. If the query is very complex or the data is very large, then Dynamo DB will not be efficient at it. Compared to partition-key look-ups, SQL queries are generally slower. In this case, the data we are using is very, very small so performance difference between the two approaches is not noticeable. 

A NoSQL solution like DynamoDB is great if the data is unstructured/semi-structured or there are no clear relations between the data. It is also a good option if the purpose is to perform many insertions/retrieve items, like in this case, where we are inserting survey submissions as participants respond to a survey in real time. The reason for this is that Dynamo DB has high **throughput**, which allows for many concurrent reads and writes. 

It is also important to mention that Dynamo DB prioritizes **availability** by default. This is a limitation because when we perform a query, we might not get back the latest version. It guarantees **eventual consistentcy**, which is good enough if are only performing a few spaced-out queries, but it is not ideal if we have many concurrent I/O's.

However, if we want to perform a lot of relatively small and fast analytical queries (like in **part b**), then a better option is a **Relational Database** solution like:

- MySQL
- PostgreSQL
- AWS Aurora

This would be a better option because relational databases are specifically optimized for small analytical SQL queries that are row-centric. Essentially, when we perform a query in a relational database, it goes over _row by row_ to complete the query. This works very well in databases that are not big data (if the data is very large, then Redshift would be the better option). Unlike DynamoDB, Relational Databases emphasize **consistency** by default, which is important when there are many concurrent reads and writes, otherwise when someone is performing a query, they might not get the most up-to-date result.

In the case that the data is very large, aka in the petabyte range, then it is preferrable to use Redshift instead of a relational database because it can handle parallel queries across a cluster of nodes and can perform faster queries by _columns_ instead of iterating over entire rows.


In [20]:
# Tear down cloud architecture
remove_objects('mariagabrielaa-a5')
delete_table(table)